# Postprocess

### 1. Merge relation

1, 6: xEffect / oEffect / objEffect

2: xAttr(feels/be), xIntent

3, 4, 8, 9: Causes (general causes)

5, 10: xWant, oWant

7: xReact/oReact

### 2. Prepare train/dev/test

In [1]:
import os
import numpy as np
import networkx as nx
from tqdm import tqdm

G_Glucose = nx.read_gpickle("../../data/glucose_data/update/G_Glucose_v2.pickle")

In [18]:
import os
import numpy as np
import networkx as nx
from tqdm import tqdm

G_Glucose = nx.read_gpickle("../../data/glucose_data/update/G_Glucose_v2.pickle")
if os.path.exists("../../data/glucose_data/update/split_v2.npy"):
  trn_idx, dev_idx, tst_idx = np.load("../../data/glucose_data/update/split_v2.npy", allow_pickle=True)
else:
  all_hids = list(set([feat["hid"] for h, t, feat in G_Glucose.edges.data()]))
  all_hids.sort()
  np.random.shuffle(all_hids)
  trn_idx, dev_idx, tst_idx = all_hids[:int(0.8*len(all_hids))],\
    all_hids[int(0.8*len(all_hids)):int(0.9*len(all_hids))],\
    all_hids[int(0.9*len(all_hids)):]
  np.save("../../data/glucose_data/update/split_v2.npy", [trn_idx, dev_idx, tst_idx])
split_dict = {}
for i in trn_idx:
  split_dict[i] = "trn"
for i in dev_idx:
  split_dict[i] = "dev"
for i in tst_idx:
  split_dict[i] = "tst"

/home/tfangaa/anaconda3/envs/torch16/lib/python3.8/site-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


In [19]:
G_glucose_new = nx.DiGraph()

for h, t, feat in G_Glucose.edges.data():
  head_subj = h.split()[0]
  tail_subj = t.split()[0]
  reverse = False 
  # As in GLUCOSE all relations are cause. We need a flag 
  # to inverse head and tail to be compatible with ATOMIC rels
  if feat["list_id"] in [1, 6]:
    new_rel = "general Effect"
    if head_subj in ["PersonX", "PeopleX"]:
      if tail_subj in ["PersonX", "PeopleX"]:
        new_rel = "xEffect"
      elif tail_subj in ["PersonY", "PersonZ", "PeopleY", "PeopleZ"]:
        new_rel = "oEffect" 
  elif feat["list_id"] == 2:
    reverse = True
    if "feel" in h:
      new_rel = "xAttr"
    else:
      new_rel = "xIntent"
  elif feat["list_id"] in [3, 4, 8, 9]:
    new_rel = "Causes"
  elif feat["list_id"] in [5, 10]:
    new_rel = "general Want"
    if head_subj in ["PersonX", "PeopleX"]:
      if tail_subj in ["PersonX", "PeopleX"]:
        new_rel = "xWant"
      elif tail_subj in ["PersonY", "PersonZ", "PeopleY", "PeopleZ"]:
        new_rel = "oWant"
  else:
    new_rel = "general React"  
    if head_subj in ["PersonX", "PeopleX"]:
      if tail_subj in ["PersonX", "PeopleX"]:
        new_rel = "xReact"
      elif tail_subj in ["PersonY", "PersonZ", "PeopleY", "PeopleZ"]:
        new_rel = "oReact"
  if reverse:
    G_glucose_new.add_edge(t, h, relation=[new_rel], split=split_dict[feat["hid"]],)
  else:
    G_glucose_new.add_edge(h, t, relation=[new_rel], split=split_dict[feat["hid"]],)

In [20]:
nx.write_gpickle(G_glucose_new, '../../data/glucose_data/update/G_Glucose_atomic_rel_v2.pickle')